In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
import holodeck.gravwaves
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode

log = holo.log
log.setLevel(logging.INFO)

# Quick-Start

Create an Illustris-Based Population, and a simple binary-evolution model

In [ ]:
# ---- Create initial population

pop = holo.population.Pop_Illustris()

# ---- Evolve binary population

# create a fixed-total-time hardening mechanism
fixed = holo.evolution.Fixed_Time.from_pop(pop, 2.0 * GYR)
# Create an evolution instance using population and hardening mechanism
evo = holo.evolution.Evolution(pop, fixed)
# evolve binary population
evo.evolve()

Calculate GWB

In [ ]:
# construct sampling frequencies
freqs = holo.utils.nyquist_freqs(dur=20.0*YR, cad=0.3*YR)
# calculate discretized GW signals
gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=30)
gwb.emit()

Plot GWB

In [ ]:
plot.plot_gwb(gwb)
plt.show()

# Step-by-Step, with additional features

Construct Illustris-Based Binary Population

In [ ]:
pop = holo.population.Pop_Illustris()
ill_name = os.path.basename(pop._fname).split('_')[1]
print("Loaded", pop.size, "binaries from Illustris", ill_name)

Plot the properties of the Illustris population at the time of 'merger' events in the cosmological simulations

In [ ]:
plot.plot_bin_pop(pop)
plt.show()

Apply a modifier to resample binary population by some factor, increasing the number of sample points

In [ ]:
mod_resamp = holo.population.PM_Resample(resample=5.0)
old_size = pop.size
pop.modify(mod_resamp)
print(f"Population size increased from {old_size} to {pop.size} elements")

Plot resampled population, now with many more points

In [ ]:
plot.plot_bin_pop(pop)
plt.show()

Apply another `Modifier` to change the MBH masses to match a particular M-Mbulge relation

In [ ]:
# Create the modifier using M-Mbulge relation
mmbulge = holo.relations.MMBulge_KH13()
mod_kh13 = holo.population.PM_Mass_Reset(mmbulge, scatter=True)

# Choose percentiles for reporting statistical properties
percs = 100*sp.stats.norm.cdf([-1, 0, 1])
percs = [0,] + percs.tolist() + [100,]

# Format nicely
str_array = lambda xx: ", ".join(["{:.2e}".format(yy) for yy in xx])
str_masses = lambda xx: str_array(np.percentile(xx/MSOL, percs))

# Modify population
print(f"                {0:7.0f}% {16:7.0f}% {50:7.0f}% {84:7.0f}% {100:7.0f}%")
print("Masses before: ", str_masses(pop.mass))
pop.modify(mod_kh13)
print("Masses after : ", str_masses(pop.mass))

Plot MBH-Galaxy scaling relationship, showing old and new masses (i.e. before and after modification)

In [ ]:
plot.plot_mbh_scaling_relations(pop)
plt.show()

# Binary Evolution Models

## Phenomenological 'Fixed Time' Evolution Model

The `holodeck.evolution.Fixed_Time` class uses a simple parametrized hardening model that forces all binaries to coalesce in a given total amount of time.  It uses a double power-law in hardening timescale [i.e. $\tau_a \equiv a / (da/dt) = dt/d\ln a$], with two power-law indices, and a transition binary-separation, as the parameters.  Based on those three parameters, it constructs a complete binary evolution history for each binary (including GW emission), and chooses a normalization such that the total evolution time matches the desired value.

Construct an `Evolution` instance using a `Fixed_Time` model for the given lifetime.

In [ ]:
# Set timescale for all binaries to merge over
fix_time = 3.0 * GYR
# ---- Construct hardening model with this desired merging time
# When this instance is created, it calculates the appropriate hardening normalization for each
# binary to reach the desired coalescing time
fixed = holo.evolution.Fixed_Time.from_pop(pop, fix_time)
# Construct evolution instance using fixed time hardening
evo = holo.evolution.Evolution(pop, fixed)
# Evolve population
evo.evolve()

Compare resulting lifetimes to the targeted lifetime.  The match won't be perfect, but it should be within a few percent of the desired timescale (`fix_time`).

In [ ]:
# Calculate the total lifetime of each binary
time = evo.tlbk
dt = time[:, 0] - time[:, -1]

# Create figure
fig, ax = plot.figax(scale='lin', xlabel='Time: actual/specified', ylabel='density')
# use kalepy to plot distribution
kale.dist1d(dt/fix_time, density=True)

plt.show()

Plot the hardening timescale ($\tau_a$) vs. binary separation ($a$).

In [ ]:
# Create spacing in separation (xaxis) to plot against
sepa = np.logspace(-4, 4, 100) * PC
# Plot hardening rates
plot.plot_evo(evo, sepa=sepa)

plt.show()

Plot the hardening timescale ($\tau_f$) vs. binary frequency ($f$).

In [ ]:
# Create frequency spacing (xaxis) to plot against
freqs = np.logspace(-3, 1, 30) / YR
# plot evolution
plot.plot_evo(evo, freqs=freqs)
plt.show()

Calculate GWB from the `evolution` instance, which takes into account the fixed-time hardening model

In [ ]:
gwb = holo.gravwaves.GW_Discrete(evo, freqs, nreals=5)
gwb.emit()

Plot GWB.

In [ ]:
plot.plot_gwb(gwb)
plt.show()

### Compare varying parameters of the hardening model, on the resulting GWB spectrum

In [ ]:
# Set timescale for all binaries to merge over
fix_time_list = np.array([2.0, 4.0]) * GYR
trans_sep_list = np.array([100.0, 10.0]) * PC

gwbs = []
labels = []
evos = []

for ii, fix in enumerate(tqdm.tqdm_notebook(fix_time_list)):
    for jj, rchar in enumerate(tqdm.tqdm_notebook(trans_sep_list, leave=False)):
        fixed = holo.evolution.Fixed_Time.from_pop(pop, fix, rchar=rchar, progress=False)
        lab = rf"$T={fix/GYR:.1f} \, \mathrm{{Gyr}}$, $R_{{c}} = {rchar/PC:.1f} \, \mathrm{{pc}}$"
        labels.append(lab)

        _evo = holo.evolution.Evolution(pop, fixed)
        _evo.evolve(progress=False)
        evos.append(_evo)
        _gwb = holo.gravwaves.GW_Discrete(_evo, freqs, nreals=10)
        _gwb.emit(progress=False)
        gwbs.append(_gwb)


In [ ]:
fig, axes = plot.figax(figsize=[10, 6], nrows=2, sharex=True, xlabel='Frequency $[1/\mathrm{yr}]$')
axes[1].set(ylabel='Characteristic Strain $h_c$')

for ii, (evo, gwb, lab) in enumerate(zip(evos, gwbs, labels)):
    plot.plot_evo(evo, freqs=freqs, ax=axes[0])
    hc = np.median(gwb.back, axis=-1)
    axes[1].plot(freqs*YR, hc, label=lab)
    
axes[1].legend()
plt.show()

# Sample Full Light-Cone Population of Binaries

In [ ]:
# pop = holo.population.Pop_Illustris()
# mod_resamp = holo.population.PM_Resample(resample=2.0)
# pop.modify(mod_resamp)

# mmbulge = holo.relations.MMBulge_MM13(mamp=7e8*MSOL)
# mod_mm13 = holo.population.PM_Mass_Reset(mmbulge, scatter=True)
# pop.modify(mod_mm13)

# fixed = holo.evolution.Fixed_Time.from_pop(pop, 2.0 * GYR)
# evo = holo.evolution.Evolution(pop, fixed)
# evo.evolve()

# fobs = holo.utils.nyquist_freqs(20.0 * YR, 0.2 * YR)

# gwb = holo.gravwaves.GW_Discrete(evo, fobs, nreals=10)
# gwb.emit()

# plot.plot_gwb(gwb)
# plt.show()



# samples = evo.sample_full_population(fobs, DOWN=None)
# hs, fo = holo.sam._strains_from_samples(samples)

# nloud = 5
# colors = plot._get_cmap('plasma')(np.linspace(0.05, 0.95, nloud))# print(colors)

# fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
# for ii in utils.tqdm(range(fobs.size-1)):
#     # if ii < 10 or ii > 16:
#     #     continue
    
#     fextr = [fobs[ii+jj] for jj in range(2)]
#     fextr = np.asarray(fextr)
#     cycles = 1.0 / np.diff(np.log(fextr))[0]

#     idx = (fextr[0] <= fo) & (fo < fextr[1])
#     hs_bin = hs[idx]
#     fo_bin = fo[idx]    

#     tot = np.sqrt(np.sum(cycles * hs_bin**2))
#     ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

#     idx = np.argsort(hs_bin)[::-1]
    
#     for jj, cc in enumerate(colors):
#         if jj > len(idx):
#             break
#         hi = idx[jj]
#         lo = idx[jj+1:]
#         gw_hi = np.sqrt(np.sum(cycles * hs_bin[hi]**2))
#         gw_lo = np.sqrt(np.sum(cycles * hs_bin[lo]**2))

#         fave = np.average(fo_bin[hi], weights=hs_bin[hi])
#         ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
#         ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

# plt.show()